<a href="https://colab.research.google.com/github/Son-github/MachineLearning_Study/blob/main/chapter5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42) #훈련 세트, 테스트 세트

In [ ]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42) #위에서 나온 훈련 세트를 다시 훈련 세트, 검증 세트로 나눔

In [ ]:
print(sub_input.shape, val_input.shape) #훈련 세트, 검증 세트

(4157, 3) (1040, 3)


In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [ ]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target) #cross_validate(평가하려는 모델, 훈련 데이터, 타겟 데이터)
print(scores) #fit_time과 score_time은 훈련하는 시간과 검증하는 시간을 뜻함. 각 키마다 5개의 숫자가 담겨있는데 그 이유는 cross_validate가 기본적으로 5-폴드 교차 검증을 수행하기 때문

{'fit_time': array([0.01162624, 0.0073669 , 0.00754261, 0.00756001, 0.00722075]), 'score_time': array([0.00154972, 0.00133872, 0.0011518 , 0.00125194, 0.00118852]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np
print(np.mean(scores['test_score'])) #교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻은 검증 폴드의 점수

0.855300214703487


In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) #앞서 수행한 교차 검증과 동일.
print(scores)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01374269, 0.00690341, 0.00642729, 0.00542521, 0.00532508]), 'score_time': array([0.00230694, 0.00144911, 0.00142646, 0.00146079, 0.00105619]), 'test_score': array([0.85769231, 0.86730769, 0.88642926, 0.85659288, 0.87969201])}
0.8695428296438884


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42) #n_splits매개변수는 몇(k) 폴드 교차 검증을 할지 정함
scores = cross_validate(dt,train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
from sklearn.model_selection import GridSearchCV #모델이 학습할 수 없어 사용자가 지정해야하만 하는 파라미터인 하이퍼파라미터 탐색과 교차 검증을 한번에 수행하는 함수
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]} #탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만듬. 여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값ㅇ르 시도.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1) #cv의 매개변수의 기본값은 5이기 때문에 5x5=25개의 모델을 훈련. n_jobs매개변수에서 병렬 실행에 사용할 cpu코어 수를 지정. 매개변수의 기본값은 1이며, -1로 지정하면 시스템에 있는 모든 코어 사용 가능

In [ ]:
gs.fit(train_input, train_target) #결정 트리 모델 min_impurity_decrease 값을 바꿔가며 총 5번 실행. 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련.

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
dt = gs.best_estimator_ #그리도 서치로 찾은 최적의 매개변수는 best_params_ 속성에 저장.
print(dt.score(train_input, train_target))

0.9615162593804117


In [ ]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [ ]:
print(gs.cv_results_['mean_test_score']) #각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)} #불순도 감소 최소량, 트리의 깊이, 노드를 나누기 위한 최소 샘플 수를 지정.

In [ ]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [ ]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([4, 7, 8, 7, 8, 8, 0, 1, 3, 3])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([109, 106,  95, 101, 103, 106,  80,  93, 108,  99]))

In [ ]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.69087467, 0.32137935, 0.45265252, 0.08192745, 0.5615612 ,
       0.74417663, 0.44848803, 0.81360354, 0.38335114, 0.81322092])

In [ ]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1,25),
          } #min_samples_leaf 매개변수는 리프 노드가 되기 위한 최소 샘플의 개수. 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할 안함.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42) #샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2453fcc970>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f2452c6a5e0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f24aa804e80>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f2452c52fd0>},
                   random_state=42)

In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884


In [ ]:
dt = gs.best_estimator_

In [ ]:
print(dt.score(test_input, test_target))

0.86
